# Crypto Trade Finder Notebook

## Install Python Packages 

This code block is used to install two Python packages, 'ccxt' and 'pandas_ta', for use in the Jupyter Notebook. The '!pip install' command is used to upgrade the packages if they are already installed, or to install them if they are not. The '--upgrade' flag ensures that the packages are updated to the latest version.

In [ ]:
!pip install --upgrade ccxt pandas_ta

## Import Python Libraries and Modules

These code lines import various libraries and modules needed to perform cryptocurrency technical analysis. These include:

    "ccxt" for fetching data from cryptocurrency exchanges
    "datetime" for handling date and time values
    "math" for mathematical operations
    "pandas" for handling and analyzing data in tabular form
    "pandas_ta" for performing technical analysis on financial data
    "numpy" for numerical computations
    "matplotlib.pyplot" for visualizing data and results
    "warnings" for handling warning messages.

These libraries are used in the subsequent code blocks to fetch cryptocurrency data, perform technical analysis on it, and visualize the results.

In [ ]:
import ccxt
from datetime import datetime, timedelta, timezone
import math
import pandas as pd
import pandas_ta as ta
import numpy as np
import matplotlib.pyplot as plt
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)

## Variables 

This section of the Jupyter Notebook sets the variables for the cryptocurrency market analysis. The following variables are defined:

    MARKET: This variable specifies the cryptocurrency market to be analyzed, given as the trading pair (e.g. "KDA/USDT").
    PERIOD: This variable specifies the time period for the analysis, given as either '1m' for one minute or '1h' for one hour.
    EXCHGE: This variable specifies the exchange from which data will be fetched, given as the exchange's name (e.g. "bittrex").
    profit_percent: This variable specifies the minimum profit percentage desired from the trade, given as a decimal value (e.g. 0.025 for 2.5%).
    loss_percent: This variable specifies the maximum loss percentage desired from the trade, given as a decimal value (e.g. 0.025 for 2.5%).
    max_count: This variable specifies the maximum number of trades to consider, given as an integer value (e.g. 24).
    rsi_min: This variable specifies the minimum Relative Strength Index (RSI) value to consider, given as a decimal value (e.g. 40).

To modify the analysis parameters, simply change the values of these variables to your desired values.


In [ ]:
MARKET="KDA/BUSD"
PERIOD="5m" # '1m', 1h
EXCHGE="binance"
profit_percent = float(0.025)
loss_percent = float(0.025)
max_count=24
rsi_min=float(40)

## Pandas-TA Help (Optional Step)

The code block provides information and documentation on the ta library, a technical analysis library for financial markets. The first line creates an empty DataFrame named df.

The second line uses the help function to display information about the ta extension. The ta extension provides technical analysis functionality for financial data stored in a DataFrame.

The third line uses the df.ta.indicators() method to list all the available technical indicators in the ta library.

The last line uses the help function to display information about a specific technical indicator named bbands. The bbands function calculates Bollinger Bands, a type of moving average envelope used in technical analysis.


In [ ]:
# Help about this, 'ta', extension
df = pd.DataFrame()
help(df.ta)

# List of all indicators
df.ta.indicators()

# Help about an indicator such as bbands
help(ta.bbands)

## Get the Market Data

This code block verifies the availability of data from a specified exchange and market. It uses a library called ccxt to retrieve the data and specifies the time period. If the exchange, market or time period is not available, the code will display an error message and stop the program. If the data is retrieved successfully, it is stored in a pandas data frame with the date column converted to a datetime object and set as the index.

In [ ]:
try:
    exchange = getattr (ccxt, EXCHGE) ()
except AttributeError:
    print('-'*36,' ERROR ','-'*35)
    print('Exchange "{}" not found. Please check the exchange is supported.'.format(EXCHGE))
    print('-'*80)
    quit()
 
# Check if fetching of OHLC Data is supported
if exchange.has["fetchOHLCV"] != True:
    print('-'*36,' ERROR ','-'*35)
    print('{} does not support fetching OHLC data. Please use another exchange'.format(EXCHGE))
    print('-'*80)
    quit()
 
# Check requested timeframe is available. If not return a helpful error.
if (not hasattr(exchange, 'timeframes')) or (PERIOD not in exchange.timeframes):
    print('-'*36,' ERROR ','-'*35)
    print('The requested timeframe ({}) is not available from {}\n'.format(PERIOD,EXCHGE))
    print('Available timeframes are:')
    for key in exchange.timeframes.keys():
        print('  - ' + key)
    print('-'*80)
    quit()
 
# Check if the symbol is available on the Exchange
exchange.load_markets()
if MARKET not in exchange.symbols:
    print('-'*36,' ERROR ','-'*35)
    print('The requested symbol ({}) is not available from {}\n'.format(MARKET, EXCHGE))
    print('Available symbols are:')
    for key in exchange.symbols:
        print('  - ' + key)
    print('-'*80)
    quit()

# Get data
data = exchange.fetch_ohlcv(MARKET, PERIOD)
header = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
df = pd.DataFrame(data, columns=header)
df.Date = pd.to_datetime(df['Date'], unit='ms')
df.set_index('Date', inplace=True) 

## Add Indicators

The code block uses the ta library to calculate several technical indicators based on the close prices of the crypto currency and adds these values as new columns to the dataframe. The indicators include Moving Average Convergence Divergence (MACD), Exponential Moving Average (EMA) and Simple Moving Average (SMA) of various lengths, Relative Strength Index (RSI), Chande Momentum Oscillator (CMO), Accumulation/Distribution Oscillator (ADOSC), Kaufman Adaptive Moving Average (KAMA), and Balance of Power (BOP). You can add additional indicators by using the help section above to find additional indicators and then defining them in the code below.

In [ ]:
MACD = ta.macd(close=df['Close'], window_slow = 26, window_fast = 12)
df = pd.concat([df, MACD], axis=1).reindex(df.index)
df['EMA24'] = ta.ema(df['Close'], length=24)
df['SMA24'] = ta.sma(df['Close'], length=24)
df['EMA48'] = ta.ema(df['Close'], length=48)
df['SMA48'] = ta.sma(df['Close'], length=48)
df['EMA200'] = ta.ema(df['Close'], length=200)
df['SMA200'] = ta.sma(df['Close'], length=200)
df['RSI-21'] = ta.rsi(close=df['Close'], length=21)
df['RSI-14'] = ta.rsi(close=df['Close'], length=14)
df['RSI-9'] = ta.rsi(close=df['Close'], length=9)
df['RSI-6'] = ta.rsi(close=df['Close'], length=6)
df['CMO-6'] = ta.cmo(close=df['Close'], length=6, talib = False)
df['CMO-9'] = ta.cmo(close=df['Close'], length=9, talib = False)
df['CMO-14'] = ta.cmo(close=df['Close'], length=14, talib = False)
df['CMO-21'] = ta.cmo(close=df['Close'], length=21, talib = False)
df['ADOSC'] = ta.adosc(high=df['High'], low=df['Low'], close=df['Close'], volume=df['Volume'], fastperiod=6, slowperiod=20)
df['KAMA'] = ta.kama(close=df['Close'])
df['BOP'] = ta.bop(open_=df['Open'], high=df['High'], low=df['Low'], close=df['Close'])

## Display Indicators and Graphs

This code creates a visualization for the columns of a dataframe in Python. The visualization is created using the matplotlib library and the style of the plot is set to '_classic_test_patch'. The visualization consists of multiple subplots, where the number of subplots is determined by the number of columns in the dataframe, and each subplot shows a plot of a column of the dataframe. The title and y-axis label of each subplot is set to the name of the column it is plotting. Finally, the visualization is displayed using the 'plt.show()' function.

In [ ]:
plt.style.use('_classic_test_patch')

fig, axs = plt.subplots(math.ceil(df.shape[1]/3), 3, sharex=True, figsize=(25, 45))

axs = axs.flatten()

for i, (col, ax) in enumerate(zip(df.columns, axs)):
    df[col].plot(ax=ax)
    ax.set_title(col)
    ax.set_ylabel(col)


plt.tight_layout()
plt.show()

## Find Trades

The code initializes several columns in the data frame df with the names "Buy Signal", "Sell Signal", "Buy Price", "Sell Price", and "Trade Value". Then, it calculates buy and sell signals for a trading strategy based on a few technical indicators. The code uses these indicators to determine whether to open a trade (buy signal) or close a trade (sell signal). The code also keeps track of the buy and sell prices, and the profit/loss of each trade. The code sets different conditions for opening and closing trades such as reaching a certain profit target, reaching a loss target, having a trade open for too long, or selling at an optimal time based on market conditions.



In [ ]:
## Initialize the columns that we need
df['Buy Signal'] = np.nan
df['Sell Signal'] = np.nan
df['Buy Price'] = np.nan
df['Sell Price'] = np.nan
df['Trade Value'] = np.nan

## Calculate the buy & sell signals
open_trade=False
buy=0
count=0

for x in range(24, len(df)):
    if open_trade and df['Buy Signal'][x-1] == 1:
        df['Buy Price'][x] = df['Open'][x]
        buy = float(df['Open'][x])
        count=0
        profit_target = (float(1.0) + profit_percent) * float(buy)
        loss_target = (float(1.0) - loss_percent) * float(buy)
        print("Trade Opened @ "+str(buy))
    elif (open_trade != True) and df['Sell Signal'][x-1] == 1:
        df['Sell Price'][x] = df['Open'][x]
        df['Trade Value'][x] = float(df['Open'][x]) - buy
        print("Buy: "+str(buy)+" Sell: "+str(df['Sell Price'][x])+ " Trade P/L: "+str(df['Trade Value'][x]))
    elif (open_trade != True) and float(df['KAMA'][x]) >= float(df['KAMA'][x-2]) and float(df['KAMA'][x]) < float(df['Close'][x]) and ( float(df['RSI-14'][x-1]) < rsi_min or float(df['RSI-14'][x-2]) < rsi_min or float(df['RSI-14'][x-3]) < rsi_min or float(df['RSI-14'][x-4]) < rsi_min) and float(df['MACDh_12_26_9'][x]) > 0:
        df['Buy Signal'][x] = 1
        open_trade=True
        count=0
    # Profit Take    
    elif (open_trade == True) and (df['Close'][x] >= profit_target) and count > 0:
        df['Sell Signal'][x] = 1
        open_trade=False
        print("Profit Take")
    elif (open_trade == True) and (df['Close'][x] <= loss_target) and count > 0:
        df['Sell Signal'][x] = 1
        open_trade=False
        print("Loss Management")
    elif (open_trade == True) and (count >= max_count) and count > 0:
        df['Sell Signal'][x] = 1
        open_trade=False
        print("Trade Open Too Long")
    elif (open_trade == True) and float(df['Close'][x]) <= float(df['Close'][x-1]) and buy < float(df['Close'][x]) and count > 5:
        df['Sell Signal'][x] = 1
        open_trade=False
        print("Optimal Sale")
    count = count + 1

## Graph The Trades

This code creates a graph that displays the crypto currencies closing price, marked by the "Close" column, and the prices for buying and selling the crypto, marked by the "Buy Price" and "Sell Price" columns, respectively. Green upward-pointing triangles will show the "Buy Price" and red downward-pointing triangles will show the "Sell Price". 

In [ ]:
## Chart the buy/sell signals:
plt.style.use('_classic_test_patch')
fig, ax = plt.subplots(sharex=True, figsize=(25,15))
## Chart the stock close price & buy/sell signals:
ax.scatter(df.index, df['Buy Price'],  color = 'green',  marker = '^', alpha = 1)
ax.scatter(df.index, df['Sell Price'],  color = 'red',  marker = 'v', alpha = 1)
ax.plot(df['Close'], alpha = 0.8)
ax.grid()